In [1]:
!conda env list

# conda environments:
#
base                     /home/chris/anaconda3
basic                    /home/chris/anaconda3/envs/basic
basic (copy)             /home/chris/anaconda3/envs/basic (copy)
torch                    /home/chris/anaconda3/envs/torch
torch (copy)             /home/chris/anaconda3/envs/torch (copy)
trans                    /home/chris/anaconda3/envs/trans
trans (copy)             /home/chris/anaconda3/envs/trans (copy)
trans2                *  /home/chris/anaconda3/envs/trans2



In [2]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses urllib3 pandas pytorch-transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [4]:
!git clone https://github.com/e9t/nsmc.git

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [5]:
train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [6]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df = train_df.sample(frac=0.4, random_state=999)
test_df = test_df.sample(frac=0.4, random_state=999)

In [7]:
class NsmcDataset(Dataset):
    """ Naver Sentiment Movie Corpus Dataset """
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

In [8]:
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True, num_workers=2)

In [9]:
device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)

100%|██████████| 714314041/714314041 [03:44<00:00, 3187450.28B/s] 


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [10]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

model.train()
for epoch in range(epochs):

    for text, label in train_loader:
        optimizer.zero_grad()

        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

<ipython-input-10-90a9cf38d465>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(label)
<ipython-input-10-90a9cf38d465>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.argmax(F.softmax(logits), dim=1)


[Epoch 1/1] Iteration 500 -> Train Loss: 0.6947, Accuracy: 0.514
[Epoch 1/1] Iteration 1000 -> Train Loss: 0.6926, Accuracy: 0.521
[Epoch 1/1] Iteration 1500 -> Train Loss: 0.6941, Accuracy: 0.516
[Epoch 1/1] Iteration 2000 -> Train Loss: 0.6938, Accuracy: 0.514
[Epoch 1/1] Iteration 2500 -> Train Loss: 0.6806, Accuracy: 0.571
[Epoch 1/1] Iteration 3000 -> Train Loss: 0.6131, Accuracy: 0.662
[Epoch 1/1] Iteration 3500 -> Train Loss: 0.6011, Accuracy: 0.687
[Epoch 1/1] Iteration 4000 -> Train Loss: 0.6043, Accuracy: 0.671
[Epoch 1/1] Iteration 4500 -> Train Loss: 0.5737, Accuracy: 0.684
[Epoch 1/1] Iteration 5000 -> Train Loss: 0.5600, Accuracy: 0.710
[Epoch 1/1] Iteration 5500 -> Train Loss: 0.5186, Accuracy: 0.749
[Epoch 1/1] Iteration 6000 -> Train Loss: 0.5373, Accuracy: 0.738
[Epoch 1/1] Iteration 6500 -> Train Loss: 0.5183, Accuracy: 0.731
[Epoch 1/1] Iteration 7000 -> Train Loss: 0.5079, Accuracy: 0.739
[Epoch 1/1] Iteration 7500 -> Train Loss: 0.4959, Accuracy: 0.773
[Epoch 1/1]

In [11]:
# evaluation
model.eval()

nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)

<ipython-input-11-7009dc78d862>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(label)
<ipython-input-11-7009dc78d862>:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.argmax(F.softmax(logits), dim=1)


Test accuracy:  0.8091404570228511
